In [13]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import csv
import pandas as pd
import itertools
import math
import os
import sympy as sp
import random
from sklearn.model_selection import train_test_split
from sympy import symbols, Function, diff
output_directory = r'../../../Data/INTEGRAL_MANIFOLDS/Exam7_T5_N100'
output_directory1 = r'../../../results/INTEGRAL_MANIFOLDS/Exam7_T5_N100'
def normalize(vector):
    norm = np.linalg.norm(vector)
    if norm == 0: 
        return vector
    return vector / norm
def normalized_system1(y, t):
    x1, x2, x3 = y
    f = np.array([0, -x3, x2])
    return normalize(f)
def normalized_system2(y, t):
    x1, x2, x3 = y
    f = np.array([x3, 0, -x1])
    return normalize(f)
np.random.seed(42)
num_trajectories = 5
initial_conditions = np.random.uniform(low=-0.4, high=0.4, size=(num_trajectories, 3))
t = np.linspace(0, 10, 100)
trajectory_data_system1 = []
trajectory_data_system2 = []
for r in range(num_trajectories):
    y0 = initial_conditions[r]
    sol1 = odeint(normalized_system1, y0, t)
    trajectory_data_system1.append(sol1)
    sol2 = odeint(normalized_system2, y0, t)
    trajectory_data_system2.append(sol2)
with open('50.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['x1', 'x2', 'x3', 'trajectory'])
    for r in range(num_trajectories):
        for j in range(len(t)):
            writer.writerow([trajectory_data_system1[r][j, 0], 
                             trajectory_data_system1[r][j, 1], 
                             trajectory_data_system1[r][j, 2], 
                             r + 1])
        for j in range(len(t)):
            writer.writerow([trajectory_data_system2[r][j, 0], 
                             trajectory_data_system2[r][j, 1], 
                             trajectory_data_system2[r][j, 2],  
                             r + 1])
fig1, ax1 = plt.subplots(figsize=(12, 8))
fig1.suptitle('Time Evolution for System 1')
for r in range(num_trajectories):
    sol1 = np.array(trajectory_data_system1[r])
    ax1.plot(t, sol1[:, 0], label=f'Traj {r+1} (x1)', linestyle='-', alpha=0.7)
    ax1.plot(t, sol1[:, 1], label=f'Traj {r+1} (x2)', linestyle='--', alpha=0.7)
    ax1.plot(t, sol1[:, 2], label=f'Traj {r+1} (x3)', linestyle=':', alpha=0.7)
ax1.set_xlabel('Time')
ax1.set_ylabel('State Variables')
ax1.legend(loc='upper right', fontsize='small')
ax1.grid(True)
file_path1 = os.path.join(output_directory1, 'system1_trajectories.png')
plt.savefig(file_path1)
plt.close()
fig2, ax2 = plt.subplots(figsize=(12, 8))
fig2.suptitle('Time Evolution for System 2')
for r in range(num_trajectories):
    sol2 = np.array(trajectory_data_system2[r])
    ax2.plot(t, sol2[:, 0], label=f'Traj {r+1} (x1)', linestyle='-', alpha=0.7)
    ax2.plot(t, sol2[:, 1], label=f'Traj {r+1} (x2)', linestyle='--', alpha=0.7)
    ax2.plot(t, sol2[:, 2], label=f'Traj {r+1} (x3)', linestyle=':', alpha=0.7)
ax2.set_xlabel('Time')
ax2.set_ylabel('State Variables')
ax2.legend(loc='upper right', fontsize='small')
ax2.grid(True)
file_path2 = os.path.join(output_directory1, 'system2_trajectories.png')
plt.savefig(file_path2)
plt.close()
def split_data():
    trajectories = {}
    column_names = None
    with open('../../../Data/INTEGRAL_MANIFOLDS/Exam7_T5_N100/trainingp_data50.csv', 'r') as trainfile:
        reader = csv.DictReader(trainfile)
        column_names = reader.fieldnames
        for row in reader:
            trajectory = float(row['trajectory'])
            if trajectory not in trajectories:
                trajectories[trajectory] = []
            trajectory_data = {key: float(value) for key, value in row.items()}
            trajectories[trajectory].append(trajectory_data)
    for traj_points in trajectories.values():
        random.shuffle(traj_points)
    num_points_per_file = len(next(iter(trajectories.values()))) // 5  # divide into five splits (n stratify)
    for i in range(5):  # Five-fold cross-validation
        output_filename = f'B50{i+1}.csv'
        with open(os.path.join(output_directory, output_filename), 'w', newline='') as output_file:
            writer = csv.DictWriter(output_file, fieldnames=column_names)
            writer.writeheader()
            for trajectory, points in trajectories.items():
                for point in points[i * num_points_per_file: (i + 1) * num_points_per_file]:
                    writer.writerow(point)
if __name__ == "__main__":
    data = np.genfromtxt('50.csv', delimiter=',', names=True)
    training_data = []
    holdout_data = []
    for r in range(1, 6):  # this represents the number of initial data is 5. i.e., (1,6) means 5 initial data
        trajectory_subset = data[data['trajectory'] == r]
        train_set, holdout_set = train_test_split(trajectory_subset, test_size=0.2, random_state=42)
        training_data.extend(train_set)
        holdout_data.extend(holdout_set)
    column_names = data.dtype.names
    with open(os.path.join(output_directory, 'trainingp_data50.csv'), 'w', newline='') as trainfile:
        writer = csv.writer(trainfile)
        writer.writerow(column_names)
        for row in training_data:
            writer.writerow([row[col] for col in column_names])
    with open(os.path.join(output_directory, 'holdoutp_data50.csv'), 'w', newline='') as holdfile:
        writer = csv.writer(holdfile)
        writer.writerow(column_names)
        for row in holdout_data:
            writer.writerow([row[col] for col in column_names])
    split_data()
input_directory = r'../../../System/INTEGRAL_MANIFOLDS/Exam7_T5_N100'
output_directory = r'../../../Data/INTEGRAL_MANIFOLDS/Exam7_T5_N100/Traj_data'
df14 = pd.read_csv(os.path.join(input_directory, '50.csv')) # 50.csv, saved names of the data
Br = pd.read_csv('../../../System/INTEGRAL_MANIFOLDS/Exam7_T5_N100/50.csv')
tr = Br.groupby('trajectory').size()
re1 = int(round(tr.mean()))
rows_per_file = re1 
num_files = len(df14) // rows_per_file
data_chunks = [df14.iloc[i * rows_per_file:(i + 1) * rows_per_file].iloc[:, :-1] for i in range(num_files)]
if not os.path.exists(output_directory):
    os.makedirs(output_directory)
for i, chunk in enumerate(data_chunks):
    chunk.to_csv(os.path.join(output_directory, f'tr_{i + 1}.csv'), index=False)